In [143]:
# disable future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# safe imports
import os, sys       # arcgis comes with these
import datetime      # arcgis comes with this
import numpy as np   # arcgis comes with this
import pandas as pd  # arcgis comes with this

# risk imports (non-native to arcgis)
try:
    import xarray as xr  # not in arcgis
except:
    arcpy.AddError('Python library Xarray is not installed.')
    raise

# import tools
try:
    # shared folder
    sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\shared')
    sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared')
    import arc, satfetcher, tools  

    # module folder
    sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\modules')
    sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules')
    import vegfrax, cog  

except:
    arcpy.AddError('Could not find tenement tools python scripts (modules, shared).')
    raise

In [175]:
out_nc = r"C:\Users\262272G\Desktop\test\vvv.nc"
ds_vegfrax = xr.open_dataset(out_nc).compute()
in_add_result_to_map = True

ds_ls = xr.open_dataset(r"C:\Users\262272G\Desktop\test\out_ls.nc").compute()

ds_phenos = xr.open_dataset(r"C:\Users\262272G\Desktop\test\phenos_num.nc").compute()

In [199]:
ds_vegfrax['spatial_ref'].attrs = 

In [214]:
None is ds_vegfrax.get('spatial_ref2')

True

In [207]:
{} is None

False

In [21]:
c

'1'